# 1. Import Required Packages

In [1]:
import mne
import torch
import os
from pathlib import Path 
import logging
import random
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mne.preprocessing import annotate_muscle_zscore
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import tfr_morlet
import math
from mne.preprocessing import (create_eog_epochs, create_ecg_epochs,
                               corrmap)
matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

# Set seed
#random.seed(42) 

# 2. Load Raw EEG Data

In [6]:
epochDuration = 1

study_epochs = {
    'td': [],
    'asd': []
}

label_ids = {
    "asd": 1,
    "td": 2
}


data_path = Path("data/")
raw_eeg_path = data_path / "raw_eeg"

# Assumes file is formatted as {ID}_{type}_{XXXHz}.{extension}. Example TD100_raw_512Hz.asc
def csvToRaw(file, fMax=40):
    print(f"Processing {file.stem}")
    data = pd.read_csv(file, sep='\t')

    try:
        data =  data.drop(['VEOG - LkE', 'HEOG - LkE', 'Unnamed: 34'], axis=1)
    except:
        print("No EOG Channels found")
    # Get Channels
    channels = list(data.columns)

    # Format Channel names
    f = lambda str: str.split("-")[0].replace(" ", "")
    channels = [f(x) for x in channels]
    channel_count = len(channels)

    # Load Data
    data = data.transpose()
    ch_types = np.full((channel_count), "eeg")
    sfreq = int(file.stem.split("_")[2].replace("Hz", ""))
    info = mne.create_info(ch_names = channels, sfreq = sfreq, ch_types=ch_types)
    raw = mne.io.RawArray(data, info)

    # Format data date for annotations later
    raw.set_meas_date(0)
    raw.set_montage("standard_1020")

    # Convert from uV to V for MNE
    raw.apply_function(lambda x: x * 1e-6)

    # Mark bad data
    # Addressing this later now
    markMuscleArtifacts(raw, 2)

    filtered = raw.copy().filter(l_freq=1.0, h_freq=fMax)

    return raw, filtered

# Find bad spans of data using mne.preprocessing.annotate_muscle_zscore
def markMuscleArtifacts(raw, threshold, plot=False):
    #print("markMuscleArtifacts")
    threshold_muscle = threshold  # z-score
    annot_muscle, scores_muscle = annotate_muscle_zscore(
    raw, ch_type="eeg", threshold=threshold_muscle, min_length_good=0.2,
    filter_freq=[0, 60])
    raw.set_annotations(annot_muscle)

    if plot:
        fig, ax = plt.subplots()
        start = 512 * 10
        end = 512 * 20
        ax.plot(raw.times[:end], scores_muscle[:end])
        ax.axhline(y=threshold_muscle, color='r')
        ax.set(xlabel='time, (s)', ylabel='zscore', title='Muscle activity')
        plt.show()

# Reject epochs based on maximum acceptable peak-to-peak amplitude 
# https://mne.tools/stable/auto_tutorials/preprocessing/20_rejecting_bad_data.html#sphx-glr-auto-tutorials-preprocessing-20-rejecting-bad-data-py
def dropBadEpochs(epochs, plotLog=False):
    reject_criteria = dict(eeg=150e-6) # 150 µV
    flat_criteria = dict(eeg=1e-6) # 1 µV
    epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)
    if plotLog: epochs.plot_drop_log()

# Get ICs 
def runICA(epochs):
    #print("Running ICA")
    n_components = 0.99  # Should normally be higher, like 0.999!!
    method = 'picard'
    fit_params = dict(fastica_it=5)
    random_state = 42

    ica = mne.preprocessing.ICA(n_components=n_components,
        method=method,
        fit_params=fit_params,
        random_state=random_state)

    ica.fit(epochs)
    return ica

# Create epoch data, get ICs, and add to study_epochs object
def addEpochs(data, raw, start, file_length_seconds, label):
    stop = start + epochDuration
    events = mne.make_fixed_length_events(data,  label_ids[label], start=start, stop=file_length_seconds, duration=epochDuration)
    epochs = mne.Epochs(data, events, tmin=0, tmax=epochDuration, event_id={label: label_ids[label]}, baseline=(0, 0), preload=True)
    #epochs.plot(title="before")
    dropBadEpochs(epochs)
   
    # Run ICA
    ica = runICA(epochs)
    eog_epochs = create_eog_epochs(raw, "Fp1")
    eog_inds, eog_scores = ica.find_bads_eog(eog_epochs, "Fp2",  threshold=1.25)
    ica.exclude = eog_inds
    #print(eog_inds)

    # Get Clean epochs
    cleaned_epochs = ica.apply(epochs.copy())
    #cleaned_epochs.plot(title="after")
    study_epochs[label].append(cleaned_epochs)
    # update with cleaned epochs
    return epochs

# Process raw file. 
def process(file):
    raw, filtered = csvToRaw(file)
    file_length = math.floor(len(filtered.times) / float(filtered.info['sfreq']))
    #print(f"{file_length}s file length")
    label = file.parent.stem
    epochs = addEpochs(filtered, raw, 1.0, file_length, label)  # first ten seconds
    return filtered, epochs


# Clean all data in the file paths added to data_path_list
def cleanData(dir):
    data_path_list = list(raw_eeg_path.glob(f"{dir}/*/*.asc"))
    count = 0

    for file in data_path_list:
        process(file)
        count += 1

    asd_concat_epochs = mne.concatenate_epochs(study_epochs['asd'])
    td_concat_epochs = mne.concatenate_epochs(study_epochs['td'])
    print(f"{count} files processed.")
    print(f"{len(study_epochs['asd'])} asd epoch objects")
    print(f"{len(study_epochs['td'])} td td objects")
    #asd_concat_epochs.plot()
    #td_concat_epochs.plot()
    asd_concat_epochs.save(Path('out_data') / f'{dir}_asd_concat_cleaned_1_40hz_epo.fif', overwrite=True)
    td_concat_epochs.save(Path('out_data') / f'{dir}_td_concat_cleaned_1_40hz_epo.fif', overwrite=True)
    train_all_epochs = mne.concatenate_epochs([asd_concat_epochs, td_concat_epochs])
    train_all_epochs.equalize_event_counts(train_all_epochs.event_id)
    train_all_epochs.save(Path('out_data') / f'{dir}_all_epo.fif')

    
cleanData("test")
# processRandomFile()
# icaClean()


Processing TD109_raw_512Hz


C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:95: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)
C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:114: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  cleaned_epochs = ica.apply(epochs.copy())


Processing TD108_raw_512Hz


C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:95: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)
C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:114: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  cleaned_epochs = ica.apply(epochs.copy())


Processing A109_raw_512Hz


C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:95: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)
C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:114: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  cleaned_epochs = ica.apply(epochs.copy())


Processing A105_raw_512Hz


C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:95: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)
C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:114: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  cleaned_epochs = ica.apply(epochs.copy())
C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:139: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  asd_concat_epochs = mne.concatenate_epochs(study_epochs['asd'])
C:\Users\12052\AppData\Local\Temp\ipykernel_13316\2134379712.py:140: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will

4 files processed.
2 asd epoch objects
2 td td objects


# 3. Additional Helper Functions

In [3]:
def icaClean():
    asd_epochs = mne.read_epochs(Path('out_data') / 'asd_concat_cleaned_1_40hz_epo.fif')
    td_epochs = mne.read_epochs(Path('out_data') / 'td_concat_cleaned_1_40hz_epo.fif')
    #asd_epochs.plot(title="ASD")
    #td_epochs.plot(title="TD")
    ica = runICA(asd_epochs, "ASD")
    ica.plot(title="ASD")

def checkPath(dir):
    # If the image folder doesn't exist, download it and prepare it... 
    if not data_path.is_dir():
        logging.error(f"{data_path} directory DOES NOT exists.")

def walkThroughDir(dir_path):
    checkPath(dir_path)
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} file(s) in '{dirpath}'.")

def createEvent(onset, label, sFreq):
    return [onset * sFreq, 0, label]


def getRandomFile(data_list):
    random_raw_eeg_path = random.choice(data_list)
    label = random_raw_eeg_path.parent.stem
    return random_raw_eeg_path, label

def processRandomFile():
    data_path_list = list(raw_eeg_path.glob("train/*/*.asc"))
    eeg_path, label = getRandomFile(data_path_list)
    filtered, epochs = process(eeg_path)
    #epochs.compute_psd().plot()


# 4. (Example) Load and Combine Existing Epochs

In [5]:
    test_asd_epochs = mne.read_epochs(Path('out_data') / 'test_asd_concat_cleaned_1_40hz_epo.fif')
    test_td_epochs = mne.read_epochs(Path('out_data') / 'test_td_concat_cleaned_1_40hz_epo.fif')
    test_all_epochs = mne.concatenate_epochs([test_asd_epochs, test_td_epochs])

    # Set if you want to equalize the event counts
    test_all_epochs.equalize_event_counts(test_all_epochs.event_id)
    
    test_all_epochs.save(Path('out_data') / 'test_all_cleaned_equal_1_40hz_epo.fif')


FileNotFoundError: File does not exist: \\wsl$\Ubuntu-20.04\home\csc\Projects\eeg_analysis\eeg_analysis2\out_data\test_asd_concat_cleaned_1_40hz_epo.fif

# 4. (Example) Create and Save Combined Raws and Process

In [ ]:
def combineRaws():
    data_path_list = list(raw_eeg_path.glob("train/*/*.asc"))
    raws = {
        "td":[],
        "asd":[]
    }
    
    count = 0

    for file in data_path_list:
        raw, filtered = csvToRaw(file)
        file_length = math.floor(len(filtered.times) / float(filtered.info['sfreq']))
        raws[file.parent.stem].append(raw)
        print(f"{file_length}s file length - {file.parent.stem}")

    
    print(f"{len(raws['asd'])} ASD raws. ")
    print(f"{len(raws['td'])} td raws. ")

    combined_raw_asd = concatenate_raws(raws['asd'])
    combined_raw_asd.save(Path('out_data') / 'raw_asd_combined_eeg.fif', overwrite=True)
    #combined_raw_asd.plot()

    combined_raw_td = concatenate_raws(raws['td'])
    combined_raw_td.save(Path('out_data') / 'raw_td_combined_eeg.fif', overwrite=True)
    #combined_raw_td.plot()

#combineRaws()